# Package

In [1]:
import pandas as pd
import numpy as np
#from sklearn.neighbors import NearestNeighbors
from numpy import array
from numpy.linalg import norm

from operator import itemgetter


# Functions


In [22]:
def movieId_by_urserId(userid,df):
    """
    Input
    -----
        userid:int
        df:dataframe of train/test rating csv
    Return
    -----
    a list of movie id that rated by given user"""
    return list(df[df['userId'] == userid]['movieId'])

In [168]:
def Average(lst): 
    return sum(lst) / len(lst) 
  

In [3]:
def search_rating_user_id(userid,movieid):
    """
    Input
    ---
    userid:int, represent userid
    movieid:int, represent movieid (closest id that rated by given user)"""
    return rating[(rating['userId'] == userid) & (rating['movieId'] == movieid)]['rating']

In [56]:
def q_given_movieid(df,l_movieid):
    """
    Input
    ----
        df:q df,latent values of movies
        l_movieid:list of movie id, all element should be str 
    Return
    ----
        a dataframe that only contain given movieids"""
    
    return df[l_movieid]

In [124]:
def cosine_similarity (test_movie_id,userid):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        user_id: user id,int
    Return
    
    ----
        """
    #############
    #list of movie id for given user in train,int
    train_movieid_given_user = movieId_by_urserId(userid,data_train)
    #list of movie id for given user in train,str
    train_movieid_given_user_str = [str(val) for val in train_movieid_given_user]
    
    
    ###########
    
    
    l_latent_movie = []
    movie_cos_dic = {}
    for movie in train_movieid_given_user_str:
        #print(movie,type(movie)) #2，str

        numerator =array([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[movie]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)# cosimilarity value for test latent vs all train latent
        
        dict_movieid_cos = dict(zip(train_movieid_given_user_str,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1)) #from small to large
        
    return [ele for ele in reversed(l_sorted_s_to_l)] 

# Data

In [150]:
#result = pyreadr.read_r('../output/mat_fac.RData') 
p = pd.read_csv('../output/p_5.csv',index_col='Unnamed: 0')
#rmse = pd.read_csv('../data/rmse.csv')
q = pd.read_csv('../output/q_5.csv',index_col='Unnamed: 0') #movie latent values
#rating = pd.read_csv('../data/ml-latest-small/ratings.csv')
data_train = pd.read_csv('../output/data_train.csv',index_col='Unnamed: 0')
data_test = pd.read_csv('../output/data_test.csv',index_col='Unnamed: 0')

b_item = pd.read_csv('../output/b_item.csv',index_col='Unnamed: 0')
b_time = pd.read_csv('../output/b_time.csv',index_col='Unnamed: 0')
b_user = pd.read_csv('../output/b_user.csv',index_col='Unnamed: 0')

# Processing Data

In [32]:
data_test_sort = data_test.copy()
data_test_sort = data_test_sort.sort_values(by=['userId'])

In [20]:
#set(data_train['userId']) == set(data_test['userId']) #check, True

In [21]:
#set(rating['userId']) == set(data_test['userId']) #check, True

In [171]:
data_test_sort = data_test.copy()
data_test_sort = data_test_sort.sort_values(by=['userId'])

userId = list(set(rating['userId'])) #a list of user id, int

movie_id = list(q.columns[1:]) #list of all movie id,str

#movie_id_int = [int(val) for val in movie_id] #int form of movie_index
mu = Average(rating['rating'].values)

In [92]:
######### processing for functions
userid = 1

# #list of movie id for given user in train,int
# train_movieid_given_user = movieId_by_urserId(userid,data_train)
#list of movie id for given user in test,int
test_movieid_given_user = movieId_by_urserId(userid,data_test_sort)

#train_movieid_given_user_str = [str(val) for val in train_movieid_given_user]
test_movieid_given_user_str = [str(val) for val in test_movieid_given_user]


In [159]:
p_matrix = p.values
q_matrix = q.values
b_item = b_item.values
b_time = b_time.values
b_user = b_user.values

In [189]:
q_matrix.T.dot(p_matrix)

array([[-2.95007126e-02,  2.30956780e-01,  8.92718706e-01, ...,
        -4.31380583e-02, -2.73390879e-01, -2.01333854e-02],
       [ 3.64671896e-02, -3.08121579e-01,  7.77124376e-01, ...,
         1.32754799e-01, -9.09115093e-02,  1.96946425e-02],
       [-8.80797796e-02,  5.50184933e-01,  6.26383845e-01, ...,
        -1.88810461e-02,  1.04117262e-01,  2.26721317e-03],
       ...,
       [ 1.89599072e-02,  1.56356597e+00, -3.29287194e+00, ...,
        -1.20646103e-01, -4.79553785e-01, -5.72594665e-02],
       [ 1.79643035e-01,  4.02767215e-01, -8.58636742e-01, ...,
        -3.98898211e-01, -2.90848644e-02,  7.09376557e-02],
       [-1.72772969e-02,  2.20943038e-01, -3.99676756e-01, ...,
        -1.58657002e-01, -4.24379309e-01,  5.95392327e-02]])

In [188]:
#pd.DataFrame(q_matrix.T.dot(p_matrix)).add(mu).head()

In [194]:
b_time

array([[ 0.34932075,  0.47916303,  0.38422305, ..., -0.89512468,
         0.78542363,  0.610574  ],
       [ 0.3600134 ,  0.47337139, -0.15563344, ...,  0.98986046,
        -0.65159179, -0.25819697],
       [-0.394849  ,  0.49407745,  0.57094858, ..., -0.76955439,
        -0.39514984,  0.14465243],
       ...,
       [ 0.28673282,  0.14218881, -0.98995473, ...,  0.09146156,
        -0.0413037 , -0.88003394],
       [ 0.35449602,  0.37078636, -0.88895109, ..., -0.34614115,
        -0.43960638, -0.5087762 ],
       [ 0.22486134, -0.10823562, -0.61725883, ...,  0.31142585,
         0.61998642,  0.41885167]])

In [197]:
matrix_before_time =  (q_matrix.T.dot(p_matrix))+ mu + b_item.T + b_user
matrix_after_time = matrix_before_time.copy()

In [210]:

matrix_before_time[0][0] + b_time[0]
.at[2, 'A'

array([4.67684359, 4.80668587, 4.7117459 , ..., 3.43239816, 5.11294648,
       4.93809685])

In [220]:
matrix_after_time=matrix_after_time.astype(object)
df_mat.at[0,0]=df_mat[0][0] + b_time[0]

In [227]:
b_time[0] #row ，replace by col

array([ 0.34932075,  0.47916303,  0.38422305, ..., -0.89512468,
        0.78542363,  0.610574  ])

In [223]:
len(df_mat[0][0])

9724

In [219]:
df_mat =  pd.DataFrame(matrix_after_time)

In [225]:
pd.DataFrame(b_time)

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,0.349321,0.479163,0.384223,0.025276,0.132270,-0.051935,0.457440,0.112809,-0.511978,-0.184725,...,-0.507794,0.319298,0.173252,-0.627360,-0.086166,-0.689407,0.008973,-0.895125,0.785424,0.610574
1,0.360013,0.473371,-0.155633,0.581782,-0.246690,0.092837,-0.221314,0.378532,0.390297,0.451582,...,0.210977,0.550038,0.848716,0.702219,-0.119205,-0.369920,0.260821,0.989860,-0.651592,-0.258197
2,-0.394849,0.494077,0.570949,0.497026,-0.249280,0.892420,-0.276575,0.639915,0.352772,0.204975,...,0.906062,-0.180305,-0.069397,0.031306,0.795370,0.046137,0.832232,-0.769554,-0.395150,0.144652
3,-0.346110,-0.171295,0.487109,0.624596,-0.726050,0.196103,-0.437051,-0.427105,0.049292,-0.311221,...,-0.872550,0.755541,-0.232140,0.003097,0.847719,-0.502034,0.878776,-0.170828,0.321828,-0.956111
4,0.370191,0.344347,0.169982,0.467235,-0.178987,0.005806,-0.351358,0.720177,-0.109503,-0.307159,...,-0.123381,0.954639,0.026595,0.997348,0.399119,-0.733668,-0.540500,-0.289435,0.512542,0.525934
5,0.583784,-0.233059,0.391091,0.125686,0.579885,0.293908,-0.104944,0.346392,-0.416518,-0.030142,...,0.709110,-0.512518,0.068400,-0.348164,-0.743446,0.052942,0.827441,-0.151092,-0.913019,-0.386576
6,0.327948,0.367220,0.334555,0.505864,-0.181629,0.226257,-0.047157,0.402710,-0.949940,0.167666,...,-0.685068,-0.261192,0.110382,-0.750223,0.978918,-0.161444,-0.570561,-0.632355,0.866717,-0.620366
7,0.488663,-0.432276,0.357444,0.211898,0.550577,0.281945,-0.789377,0.483512,0.126905,0.183696,...,0.712065,0.166455,0.102194,-0.546908,0.967603,-0.327820,-0.718302,-0.160637,-0.859027,0.430072
8,-0.267608,-0.084964,0.293699,0.268860,-0.218135,0.195479,-0.352130,-0.473512,0.506445,-0.063541,...,-0.838380,0.767995,0.927049,-0.954137,-0.779321,-0.506636,-0.618626,-0.857749,0.001018,-0.314522
9,0.226984,0.062830,-0.638651,0.204099,0.357300,0.384789,-0.222638,-0.449034,0.604045,0.332681,...,0.235283,-0.141186,0.971883,-0.994033,0.693742,-0.320987,0.322979,-0.220372,0.923825,0.735387


# scrachwork

In [126]:
d = cosine_similarity(test_movieid_given_user_str[0],1)

In [127]:
d

[('1445', 0.592428177293655),
 ('3441', 0.5238184005117847),
 ('2137', 0.41007026068952807),
 ('2116', 0.370094205662481),
 ('3617', 0.3225903098519638),
 ('70', 0.31945076983818543),
 ('1777', 0.30646796985042385),
 ('1732', 0.30272091575464966),
 ('2502', 0.2886788288660969),
 ('2058', 0.2853007337903877),
 ('1473', 0.27563443996655257),
 ('2414', 0.2561078909444748),
 ('2654', 0.2527580178715182),
 ('2000', 0.2509632060976217),
 ('1029', 0.250280159014815),
 ('1025', 0.2458422204667194),
 ('223', 0.23402350976504907),
 ('1214', 0.2312097934821),
 ('2253', 0.2278804860347231),
 ('1298', 0.21881291017644047),
 ('2366', 0.21394938898080385),
 ('1073', 0.20924968179619574),
 ('235', 0.20869612789221678),
 ('673', 0.20823004901446113),
 ('1097', 0.20488626053999656),
 ('1127', 0.20242446593744576),
 ('2826', 0.19971607485135823),
 ('736', 0.19646595567173789),
 ('1348', 0.1951615061997804),
 ('590', 0.19146261511729604),
 ('2450', 0.1874394286409211),
 ('1080', 0.18554934993213865),
 ('2

In [79]:
l_latent_movie = []
for movie in train_movieid_given_user_str:
    #print(movie,type(movie)) #2，str
    
    
    
    numerator =array([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist]).dot(array([val
                  for sublist in np.asarray(q[[movie]]).tolist()
                  for val in sublist]))
    
    cosin_s = numerator/(norm([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist])*norm([val
                  for sublist in np.asarray(q[[i]]).tolist()
                  for val in sublist]))
    l_latent_movie.append(cosin_s)
    
index_min = np.argmin(l_latent_movie) #find the index of min value

In [80]:
index_min

102

In [58]:
# movie_index = list(q.columns[1:]) #list of all movie index,str
# movie_index_int = [int(val) for val in movie_index] #int form of movie_index


# Vj_index = ['1'] # list of given movie index
# movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [71]:
Vj_index = ['1'] # list of given movie index

movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [86]:
# Vj_index = test_movieid_given_user_str
# Vj_index[0] #要predict 的电影ID

In [ ]:
l_latent_movie = []
for i in movie_index_sub:
   # print(i,type(i)) 2，str

#     numerator =array([val
#                   for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[i]]).tolist()
#                   for val in sublist]))
    
#     cosin_s = numerator/(norm([val
#                   for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
#                   for val in sublist])*norm([val
#                   for sublist in np.asarray(q[[i]]).tolist()
#                   for val in sublist]))
#     l_latent_movie.append(cosin_s)
    
# index_min = np.argmin(l_latent_movie) #find the index of min value

# if int(Vj_index[0]) > index_min: 
#     index_similar = index_min
# else:
#     index_similar = index_min+1 #+1 would be the real index of that similar movie
    
# print(index_similar)

In [54]:
#q.as_matrix(columns=q.columns[1:2]).dot(q.as_matrix(columns=q.columns[2:3]))

In [123]:
l_latent_movie = []
movie_index = q.columns[1:]
# for i in range(len(q.columns[1:])):
#     print(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))
#     #j = i+1
#     l_latent_movie.append(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))

In [4]:
rmse.head()

,Unnamed: 0,train_test,par,epoch,RMSE
0,1,Train,"f = 10 , lambda = 0.01",1,0.825492
1,2,Train,"f = 20 , lambda = 0.01",1,0.792257
2,3,Train,"f = 10 , lambda = 0.1",1,0.896950
3,4,Train,"f = 20 , lambda = 0.1",1,0.859586
4,5,Test,"f = 10 , lambda = 0.01",1,1.313171


In [8]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]

In [13]:
neigh = NearestNeighbors(2, 0.4,metric='cosine')
neigh.fit(samples)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=0.4)

In [14]:
#class sklearn.neighbors.NearestNeighbors(n_neighbors=5, radius=1.0, algorithm=’auto’, leaf_size=30,
#metric=’minkowski’, p=2, metric_params=None, n_jobs=None, **kwargs)

In [15]:
neigh.predict([0,0,5])

AttributeError: 'NearestNeighbors' object has no attribute 'predict'